# Tourism

This example highlights how you can actually use a combination of deliberately extracting blank cells along with horizontal conditions to unpick complex (bad) table structures.

## Source

For this example we're extracting the table3 from an xlsx dataset dealing with travel and tourism.

In [1]:
from tidychef import acquire, preview
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/tourism.xlsx", tables="Table 3")
preview(table, bounded="A1:L11")

,A,B,C,D,E,F,G,H,I,J,K,L
1,3,UK residents' visits abroad by month,,,,,,,,,,
2,,,,,,,,,,,thousands,
3,,,All Visits,,,Area of visit,,,,,,
4,,,,,,North America,Europe,of which,,,Other countries,
5,,,,,,,,European Union,Other EU,EU15,,
6,,,NSA,SA,,NSA,NSA,NSA,NSA,NSA,NSA,
7,2018,Jan,5831.281022512688,7450,,280.1873908239189,4324.568633619735,3946.0897447298166,750.0352467745454,3190.916896810225,1226.524998069034,
8,,Feb,5071.372074023899,7250,,306.9532931971211,3704.8345323387316,3364.8630267535614,496.1049582724145,2870.168372849392,1059.584248488046,
9,,Mar,5689.649918024445,7600,,335.7319919123934,4165.622824517005,3911.1414280012004,524.9228361904585,3389.9458885875365,1188.2951015950462,
10,,Apr,8018.324302902642,7650,,515.9388604171628,6078.356023872903,5632.226496707873,767.7824728871175,4864.441054069019,1424.0294186125757,


# Stage 1

When using tidychef you typically don't need to have interim data transform stages, but to better explain this approach we'll do something similar here.

We're going to create a `TidyData` class with just observations and two interim columns formed from rows  4 and 5.

When we write the full recipe (later) **we'll do this all in one**, this step by step approach is purely to better explain the technique.

Here's the logic:

- Select row 4 cells minus the "of which" cells so we can use a closest relationship to create tc1 (temp country 1)
- Select most of row 5 as as tc2 (temp country 2).
- Combine the columns into "Country" ready for final output.

We'll preview this "half way" table to better explain the concept. When we do this for real we'll be dropping the temporary columns as part of the recipe.

In [2]:
from typing import Dict
from tidychef import acquire, preview
from tidychef.direction import up, down, right
from tidychef.output import Column, TidyData
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/tourism.xlsx", tables="Table 3")

anchor = table.re("All Visits").assert_one().label_as("Anchor Cell")

temp_country_1 = (anchor.shift(down).expand(right) - anchor.shift(down(2)).expand(right).is_not_blank().shift(up)).label_as("tc1")
temp_country_2 = anchor.shift(down(2)).expand(right).label_as("tc2")
observations = temp_country_2.shift(down(2)).expand(down).is_not_blank().label_as("Value")

preview(anchor, observations, temp_country_1, temp_country_2, bounded="A1:L11")

def combine(line: Dict[str, str]) -> str:
    # If temp column2 is populated combine both columns
    # around " - ".
    # If both are blank its All Countries.
    # Else just return temp column 1 value
    if len(line["tc2"]) > 0:
        return line["tc1"] + " - " + line["tc2"]
    if len(line["tc1"]) == 0 and len(line["tc2"]) == 0:
        return "All Countries"
    return line["tc1"]


tidy_data = TidyData(
    observations,
    Column(temp_country_1.finds_observations_closest(right)),
    Column(temp_country_2.finds_observations_directly(down)),
    Column.horizontal_condition("Country", combine)
)
tidy_data.to_csv("interim-tourism.csv")

Given I'd rather not fill the page up with a huge table I've included a truncated screenshot below, you can also view the interim table [here](./interim-tourism.csv). If you look at the "Country" column you'll notice it's fully and correctly populated.

![](./tourism-screenshot.png)

Now that we've explained unpicking that nasty visual relationship, let's put it together with the usual bag of tricks.

## Requirements

- Rows 4 and 5 combines as "Country"
- Column A as "Year"
- Column B as "Month". Im **not** doing to split off the markers as this is already quite a complex example, but you;d do so via the same technique documented for handling data markers.
- Row 6 as "Seasonal Adjustment"


In [3]:
from typing import Dict, List
from tidychef import acquire, preview
from tidychef.direction import up, down, left, right
from tidychef.output import Column, TidyData
from tidychef.selection import XlsxSelectable

tables: List[XlsxSelectable] = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/tourism.xlsx")
table = tables[3]

anchor = table.re("All Visits").assert_one().label_as("Anchor Cell")

temp_country_1 = (anchor.shift(down).expand(right) - anchor.shift(down(2)).expand(right).is_not_blank().shift(up)).label_as("tc1")
temp_country_2 = anchor.shift(down(2)).expand(right).label_as("tc2")
month = anchor.shift(left).fill(down).is_not_blank().label_as("Month")
year = anchor.shift(left(2)).fill(down).is_not_blank().label_as("Year")
seasonal_adjustment = anchor.shift(down(3)).expand(right).is_not_blank().label_as("Seasonal Adjustment")

unwanted = table.excel_ref("A").re("Latest three").assert_one().expand(right).expand(down)
observations = (temp_country_2.shift(down(2)).expand(down).is_not_blank() - unwanted).label_as("Value")

preview(anchor, observations, month, year, seasonal_adjustment, temp_country_1, temp_country_2, bounded="A1:L11")

def combine(line: Dict[str, str]) -> str:
    # If temp column2 is populated combine both columns
    # around " - ".
    # If both are blank its All Countries.
    # Else just return temp column 1 value
    if len(line["tc2"]) > 0:
        return line["tc1"] + " - " + line["tc2"]
    if len(line["tc1"]) == 0 and len(line["tc2"]) == 0:
        return "All Countries"
    return line["tc1"]


tidy_data = TidyData(
    observations,
    Column(temp_country_1.finds_observations_closest(right)),
    Column(temp_country_2.finds_observations_directly(down)),
    Column(year.finds_observations_closest(down)),
    Column(month.finds_observations_directly(right)),
    Column(seasonal_adjustment.finds_observations_directly(down)),
    Column.horizontal_condition("Country", combine),
    drop=["tc1", "tc2"]
)

tidy_data.to_csv("tourism.csv")

# Outputs

The tidy data can be [downloaded here](./tourism.csv) and a full inline preview of the tidydata generated is shown below for those people who'd prefer to scroll.

In [4]:
print(tidy_data)

Value,Year,Month,Seasonal Adjustment,Country
5831.281022512688,2018,Jan,NSA,All Countries
7450,2018,Jan,SA,All Countries
280.1873908239189,2018,Jan,NSA,North America
4324.568633619735,2018,Jan,NSA,Europe
3946.0897447298166,2018,Jan,NSA,Europe - European Union
750.0352467745454,2018,Jan,NSA,Europe - Other EU
3190.916896810225,2018,Jan,NSA,Europe - EU15
1226.524998069034,2018,Jan,NSA,Other countries
5071.372074023899,2018,Feb,NSA,All Countries
5689.649918024445,2018,Mar,NSA,All Countries
